In [2]:
pip install https://github.com/kpu/kenlm/archive/master.zip


Defaulting to user installation because normal site-packages is not writeable
  Using cached https://github.com/kpu/kenlm/archive/master.zip
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import json
import kenlm
import numpy as np
from tqdm import tqdm

# Config
INPUT_DIR = os.path.expanduser("/home/blu-bridge004/Desktop/pytorch/rupeshlangfil06_07")
OUTPUT_DIR = os.path.expanduser("/home/blu-bridge004/Desktop/pytorch/rupeshqualfilter06_07")
LM_PATH = "/home/blu-bridge004/Desktop/pytorch/wiki.arpa.bin"  # your KenLM bin file

os.makedirs(OUTPUT_DIR, exist_ok=True)

# Load language model
print("Loading KenLM model...")
model = kenlm.Model(LM_PATH)

def compute_perplexity(text):
    return 10.0 ** (-model.score(text.replace("\n", " ").strip()) / max(1, len(text.split())))

# Process each shard
for filename in tqdm(sorted(os.listdir(INPUT_DIR))):
    if not filename.endswith(".jsonl"):
        continue

    input_path = os.path.join(INPUT_DIR, filename)
    
    docs = []
    perplexities = []

    # Pass 1: collect perplexities
    with open(input_path, "r", encoding="utf-8") as f:
        for line in f:
            data = json.loads(line)
            text = data.get("text", "")
            ppl = compute_perplexity(text)
            docs.append((data, ppl))
            perplexities.append(ppl)

    # Compute thresholds
    perplexities = np.array(perplexities)
    head_thresh = np.percentile(perplexities, 10)
    tail_thresh = np.percentile(perplexities, 50)

    # Output files
    base = filename.replace(".jsonl", "")
    head_out = open(os.path.join(OUTPUT_DIR, f"{base}_head.jsonl"), "w", encoding="utf-8")
    middle_out = open(os.path.join(OUTPUT_DIR, f"{base}_middle.jsonl"), "w", encoding="utf-8")
    tail_out = open(os.path.join(OUTPUT_DIR, f"{base}_tail.jsonl"), "w", encoding="utf-8")

    # Pass 2: write categorized output
    for data, ppl in docs:
        if ppl <= head_thresh:
            head_out.write(json.dumps(data, ensure_ascii=False) + "\n")
        elif ppl >= tail_thresh:
            tail_out.write(json.dumps(data, ensure_ascii=False) + "\n")
        else:
            middle_out.write(json.dumps(data, ensure_ascii=False) + "\n")

    head_out.close()
    middle_out.close()
    tail_out.close()

    print(f"{filename}: Head ≤ {head_thresh:.2f}, Tail ≥ {tail_thresh:.2f}")


Loading KenLM model...


 50%|█████     | 1/2 [00:32<00:32, 32.70s/it]

shard_006.en.jsonl: Head ≤ 1313695.17, Tail ≥ 3108977.99


100%|██████████| 2/2 [01:03<00:00, 31.87s/it]

shard_007.en.jsonl: Head ≤ 1316810.44, Tail ≥ 3113513.43


In [5]:
import kenlm
model=kenlm.Model("/home/blu-bridge004/Desktop/pytorch/wiki.arpa.bin")

In [6]:
model.score("hello")

-11.460987091064453